# Apriori

## Importing the libraries

In [ ]:
import time
start_t = time.time()
!pip install apyori

In [ ]:
pip install memory_profiler

In [ ]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori
import string
import timeit

#Reading Data


In [ ]:
#handle, text, lang = en, retweet_count, favourite_count
data = pd.read_csv("tweets.csv")
keep_columns = ["handle", "text", "lang"]

# removing columns of data which we won't be dealing with and keeping only keep_columns
remove_columns_list = list(set(data.columns)-set(keep_columns))
data.drop(columns = remove_columns_list, inplace = True)

# keeping rows which have tweets in english
data = data[data.lang == "en"]
data.drop(columns = ["lang"], inplace=True)
data.to_csv("twitter.csv")

hillary = data[data.handle == "HillaryClinton"]
hillary.drop(columns = ["handle"], inplace=True)
hillary.to_csv("Hillary.csv")

trump = data[data.handle == "realDonaldTrump"]
trump.drop(columns = ["handle"], inplace=True)
trump.to_csv("Trump.csv")

pd.set_option('display.max_columns', None)



/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#Data Cleaning and Preprocessing

In [ ]:
# data cleaning
clean_data = ['',"will","a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "arent", "as", "at", "be", "because", "been", "before", "being","below", "between", "both", "but", "by","can", "cant", "cannot", "could", "couldnt", "did","didn't", "do", "does", "doesnt", "doing","dont", "down", "during", "each", "few", "for", "from", "further", "had", "hadnt","has", "hasnt", "have", "havent", "having","he", "hed", "hell", "hes", "her","here", "heres", "hers", "herself", "him", "himself", "his", "how", "how's", "i","id", "ill", "im", "ive", "if","in", "into", "is", "isnt", "it","its", "its", "itself", "lets", "me","more", "most", "mustnt", "my", "myself", "no", "nor", "not", "of", "off", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "shant", "she", "shed", "shell", "shes", "should", "shouldnt", "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them","themselves", "then", "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was","wasnt", "we", "wed", "well", "were", "weve", "were", "werent", "what", "whats", "when", "when's", "where", "wheres", "which","while", "who", "whos", "whom", "why","whys", "with", "wont", "would", "wouldnt","you", "youd", "youll", "youre", "youve", "your", "yours","yourself", "yourselves"]
punctuation = list(string.punctuation + "”’â€™")

def clean(x):
    # make lowercase
    x = x.lower()

    #x = x.replace("â€™","") if x.find("â€™") else x
    #x = x.replace("â†","") if x.find("â€™") else x

    # to remove punctuation
    temp = ''
    for i in x:
        if i in punctuation or i.isnumeric():
            continue
        temp += i
    x = temp

    # remove links

    if x.find("http") != -1 :
        pos = x.find("http")
        space_pos = x.find(" ", pos)
        x = x[:pos] #x[0:pos] + x[space_pos:] if space_pos != -1 else x[0:pos]

    # removing common words

    words = x.split(" ")
    temp = ""
    for i in words:
        if i not in clean_data:
            temp += i+" "

    x = temp.strip()
    return x

def preprocessing(Data):
    transactions = []
    for i,j in Data.iterrows():
        str = j.text
        str = clean(str)
        Data.at[i,'text'] = str.split(" ")
        transactions.append(str.split(" "))
    Data.drop(Data.columns[[0]], axis = 1, inplace = True)
    Data.to_csv("trial.csv")
    return transactions

data_hillary = pd.read_csv("Hillary.csv")
# data = data.head(50) # to reduce number of rows while testing
transactions_h = preprocessing(data_hillary)

data_trump = pd.read_csv("Trump.csv")
# data = data.head(50) # to reduce number of rows while testing
transactions_t = preprocessing(data_trump)

In [ ]:
def inspect(results):
  lhs=[]
  rhs=[]
  support=[]
  confidence=[]
  lifts=[]

  for i in results:
    for j in i[2]:
      lhs.append(tuple(j[0]))
      rhs.append(tuple(j[1]))
      support.append(i[1])
      confidence.append(j[2])
      lifts.append(j[3])
  return list(zip(lhs,rhs,confidence,support,lifts))
def apriori_func(transaction, min_sup, min_confi, min_lift): 
  pattern = apriori(transaction, min_support = min_sup, min_confidence = min_confi, min_lift = min_lift)
  results = pd.DataFrame(inspect(list(pattern)), columns= ['Antecedant','Consequent','Confidence','Support','Lifts'])
  return results


    


## Training the Apriori model for Hillary

In [ ]:
%memit results_Hillary = apriori_func(transactions_h,0.005, 0.7, 1)


peak memory: 235.59 MiB, increment: 13.51 MiB


## Training the Apriori model for Trump



In [ ]:
%memit results_Trump = apriori_func(transactions_t,0.005, 0.7, 1)


peak memory: 236.35 MiB, increment: 14.27 MiB


### Function to put the results into a Pandas DataFrame

In [ ]:
def inspect(results):
  lhs=[]
  rhs=[]
  support=[]
  confidence=[]
  lifts=[]

  for i in results:
    for j in i[2]:
      lhs.append(tuple(j[0]))
      rhs.append(tuple(j[1]))
      support.append(i[1])
      confidence.append(j[2])
      lifts.append(j[3])
  return list(zip(lhs,rhs,confidence,support,lifts))


### Displaying the first results coming directly from the output of the apriori function

In [ ]:

results_Hillary.to_csv("Results - Hillary.csv")

In [ ]:
results_Hillary.nlargest(n=20,columns='Lifts')

,Antecedant,Consequent,Confidence,Support,Lifts
7,"(pence,)","(mike,)",0.739130,0.005454,115.193478
6,"(mike,)","(pence,)",0.850000,0.005454,115.193478
3,"(policy,)","(foreign,)",0.718750,0.007379,67.889205
1,"(student,)","(debt,)",0.750000,0.005775,66.792857
24,"(states, president)","(united,)",1.000000,0.008341,64.937500
25,"(united, president)","(states,)",1.000000,0.008341,64.937500
11,"(states,)","(united,)",0.854167,0.013154,55.467448
12,"(united,)","(states,)",0.854167,0.013154,55.467448
9,"(returns,)","(tax,)",0.954545,0.006737,42.504545
4,"(night,)","(last,)",0.705882,0.007700,38.600619


### Displaying the results non sorted

In [ ]:

results_Trump.to_csv("Results - Trump.csv")


In [ ]:
results_Trump.nlargest(n=20,columns='Lifts')


,Antecedant,Consequent,Confidence,Support,Lifts
13,"(elizabeth,)","(warren,)",0.958333,0.007346,130.458333
14,"(warren,)","(elizabeth,)",1.000000,0.007346,130.458333
51,"(elizabeth,)","(goofy, warren)",0.916667,0.007027,130.458333
56,"(goofy, warren)","(elizabeth,)",1.000000,0.007027,130.458333
53,"(warren,)","(elizabeth, goofy)",0.956522,0.007027,130.211720
54,"(elizabeth, goofy)","(warren,)",0.956522,0.007027,130.211720
11,"(elizabeth,)","(goofy,)",0.958333,0.007346,120.021667
12,"(goofy,)","(elizabeth,)",0.920000,0.007346,120.021667
16,"(goofy,)","(warren,)",0.880000,0.007027,119.794783
17,"(warren,)","(goofy,)",0.956522,0.007027,119.794783


In [ ]:
end_t = time.time()
print(start_t)
print(end_t)
print(end_t - start_t)

1603342592.6158528
1603342601.0696192
8.453766345977783
